# Registro del modelo base de Hugging Face en MLflow
Esta sección descarga el modelo base de Hugging Face y lo registra en MLflow como 'image-to-text-base'.

In [ ]:
from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer
import mlflow
import mlflow.pytorch

In [ ]:
# Configuración
MODEL_NAME = 'nlpconnect/vit-gpt2-image-captioning'
MLFLOW_MODEL_NAME = 'image-to-text-base'
mlflow.set_tracking_uri('http://localhost:5000')

In [ ]:
# Descargar modelo base
print('Descargando modelo base de Hugging Face...')
model = VisionEncoderDecoderModel.from_pretrained(MODEL_NAME)
feature_extractor = ViTImageProcessor.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
print('Modelo base descargado.')

In [ ]:

# Registrar en MLflow
with mlflow.start_run(run_name='register-base-model') as run:
    print(f'Registrando modelo base en MLflow como {MLFLOW_MODEL_NAME}...')
    mlflow.log_param('huggingface_model', MODEL_NAME)
    mlflow.pytorch.log_model(model, 'model', registered_model_name=MLFLOW_MODEL_NAME)
    print('Modelo base registrado en MLflow.')
    # Guardar tokenizer y feature extractor localmente
    tokenizer.save_pretrained('./results/tokenizer_base')
    feature_extractor.save_pretrained('./results/feature_extractor_base')
    print('Tokenizer y feature extractor guardados.')

Descargando modelo base de Hugging Face...
Modelo base descargado.


MlflowException: API request to http://localhost:5000/api/2.0/mlflow/runs/create failed with exception HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /api/2.0/mlflow/runs/create (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000259DF360F30>: Failed to establish a new connection: [WinError 10061] No se puede establecer una conexión ya que el equipo de destino denegó expresamente dicha conexión'))

# Cargar el modelo base desde MLflow para fine-tuning
Esta celda carga el modelo base registrado en MLflow para usarlo como punto de partida en el fine-tuning.

In [ ]:
import torch
from transformers import ViTImageProcessor, AutoTokenizer

# Nombre del modelo registrado en MLflow
MLFLOW_MODEL_NAME = 'image-to-text-base'

# Cargar modelo desde MLflow
print('Cargando modelo base desde MLflow...')
model = mlflow.pytorch.load_model(f'models:/{MLFLOW_MODEL_NAME}/latest')
print('Modelo base cargado desde MLflow.')

# Cargar tokenizer y feature extractor locales (guardados previamente)
tokenizer = AutoTokenizer.from_pretrained('./results/tokenizer_base')
feature_extractor = ViTImageProcessor.from_pretrained('./results/feature_extractor_base')
print('Tokenizer y feature extractor cargados.')